<a href="https://colab.research.google.com/github/SokKanaTorajd/nlp-practices/blob/main/Coba_Tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow-gpu==2.0.0-alpha0

In [ ]:
import tensorflow as tf

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import operator
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers

%matplotlib inline
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

print("Device name: {}".format((x.device)))

Is there a GPU available: 
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
Is the Tensor on GPU #0:  
True
Device name: /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/") 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
ls

ls: cannot access 'drive': Transport endpoint is not connected
drive/  gdrive/  sample_data/


In [ ]:
cd gdrive/MyDrive/Colab\ Notebooks/data

/content/gdrive/MyDrive/Colab Notebooks/data


In [ ]:
import pandas as pd
df  = pd.read_csv("Indonesian Sentiment Twitter Dataset Labeled-Ok.csv")
df_test  = pd.read_csv("Unlabeled.csv")

In [ ]:
X_train, X_test  = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
y_train, y_test = X_train['sentimen'].values, X_test['sentimen'].values

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [ ]:
X_train = X_train['Tweet'].fillna('_NA_').values
X_test = X_test['Tweet'].fillna('_NA_').values
# X_submission = df_test['Tweet'].fillna('_NA_').values

In [ ]:
X_train.shape

(9725,)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
# X_submission = tokenizer.texts_to_sequences(X_submission)

In [ ]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
# X_submission = pad_sequences(X_submission, maxlen=maxlen)

In [ ]:
# def data_prep(df):
#     print("Splitting dataframe with shape {} into training and test datasets".format(df.shape))
#     X_train, X_test  = train_test_split(df, test_size=0.1, random_state=2019)
#     y_train, y_test = X_train['target'].values, X_test['target'].values
    
#     print("Filling missing values")
#     X_train = X_train['Tweet'].fillna('_NA_').values
#     X_test = X_test['Tweet'].fillna('_NA_').values
#     X_submission = df_test['Tweet'].fillna('_NA_').values
    
#     print("Tokenizing {} questions into words".format(df.shape[0]))
#     tokenizer = Tokenizer(num_words=max_features)
#     tokenizer.fit_on_texts(list(X_train))
#     X_train = tokenizer.texts_to_sequences(X_train)
#     X_test = tokenizer.texts_to_sequences(X_test)
#     X_submission = tokenizer.texts_to_sequences(X_submission)
    
#     print("Padding sequences for uniform dimensions")
#     X_train = pad_sequences(X_train, maxlen=maxlen)
#     X_test = pad_sequences(X_test, maxlen=maxlen)
#     X_submission = pad_sequences(X_submission, maxlen=maxlen)
    
#     print("Completed data preparation, returning training, test and submission datasets, split as dependent(X) and independent(Y) variables")
    
#     return X_train, X_test, y_train, y_test, X_submission

In [ ]:
# X_train, X_test, y_train, y_test, X_submission = data_prep(df)

In [ ]:
model1 = Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          440320    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2

In [ ]:
%time model1.fit(X_train, y_train, batch_size=512, epochs=5, validation_data=(X_test, y_test), verbose = 1)

Epoch 1/5
19/19 [==============================] - 6s 188ms/step - loss: 0.1975 - accuracy: 0.4845 - val_loss: -0.3703 - val_accuracy: 0.5032
Epoch 2/5
19/19 [==============================] - 3s 152ms/step - loss: -0.3971 - accuracy: 0.4918 - val_loss: -0.6544 - val_accuracy: 0.5032
Epoch 3/5
19/19 [==============================] - 3s 155ms/step - loss: -1.2319 - accuracy: 0.4918 - val_loss: -2.5126 - val_accuracy: 0.5032
Epoch 4/5
19/19 [==============================] - 3s 151ms/step - loss: -8.4475 - accuracy: 0.4918 - val_loss: -8.8609 - val_accuracy: 0.5032
Epoch 5/5
19/19 [==============================] - 3s 151ms/step - loss: -27.9243 - accuracy: 0.4918 - val_loss: -23.1804 - val_accuracy: 0.5032
CPU times: user 13.1 s, sys: 618 ms, total: 13.7 s
Wall time: 18 s


In [ ]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [ ]:
vocab = build_vocab(df['Tweet'])

In [ ]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding="utf8") if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [ ]:
glove = 'glove_wiki_id_50.txt'

print("Extracting GloVe embedding")
embed_glove = load_embed(glove)

Extracting GloVe embedding


In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 48.49% of vocab
Found embeddings for  82.76% of all text


In [ ]:
type(embed_glove)
dict(list(embed_glove.items())[20:22])

{'Indonesia': array([ 5.9440e-03,  7.0470e-03, -3.5600e-04,  9.0490e-03, -9.1120e-03,
        -7.5050e-03, -8.8590e-03,  4.6680e-03, -2.2780e-03, -2.6630e-03,
         2.8650e-03, -7.5980e-03,  2.2630e-03, -8.7430e-03,  1.9200e-03,
        -6.6060e-03,  5.4140e-03,  5.2000e-05, -5.9670e-03, -2.8960e-03,
        -8.8280e-03,  9.4330e-03, -4.6000e-04,  4.0110e-03, -1.1350e-02,
        -6.1500e-03, -1.5989e-02, -6.7670e-03,  5.0100e-03,  1.6000e-03,
         7.9230e-03, -1.5841e-02, -1.1521e-02,  1.8191e-02,  9.1500e-04,
        -1.9600e-03,  1.3045e-02,  7.4020e-03, -1.3450e-03,  2.9710e-03,
        -6.4790e-03,  6.8580e-03,  5.9980e-03, -7.9770e-03,  1.5070e-02,
        -1.8128e-02, -3.8260e-03,  3.2060e-03, -1.0293e-02, -7.3240e-03],
       dtype=float32),
 'Pada': array([ 0.001962, -0.007293, -0.018094,  0.009985,  0.01025 ,  0.013142,
        -0.007931, -0.011334,  0.010864,  0.008238, -0.007061,  0.011391,
         0.014036,  0.002048,  0.002705,  0.003668, -0.00704 ,  0.003804,
   

Lower Casing Teks

In [ ]:
df['processed_tweet'] = df['Tweet'].apply(lambda x: x.lower())

In [ ]:
vocab_low = build_vocab(df['processed_tweet'])

In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)

Glove : 
Found embeddings for 47.81% of vocab
Found embeddings for  82.75% of all text


In [ ]:
oov_glove[1:20]

[('1', 197),
 ('3', 138),
 ('tengok', 136),
 ('i', 131),
 ('giveaway', 115),
 ('a', 113),
 ('mcm', 104),
 ('d', 94),
 ('hahaha', 92),
 ('k', 92),
 ('t', 89),
 ('korang', 88),
 ('m', 79),
 ('b', 77),
 ('wkwk', 75),
 ('4', 75),
 ('gimana', 73),
 ('haha', 72),
 ('5', 72)]

In [ ]:
def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")

In [ ]:
print("Glove : ")
add_lower(embed_glove, vocab)

Glove : 
Added 44 words to embedding


In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)

Glove : 
Found embeddings for 48.00% of vocab
Found embeddings for  82.96% of all text


Removing special character

In [ ]:
punctuations = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [ ]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in punctuations:
        x = x.replace(punct, '')
    return x

In [ ]:
df["processed_tweet"] = df["processed_tweet"].progress_apply(lambda x: clean_text(x))

100%|██████████| 10806/10806 [00:00<00:00, 75982.90it/s]


In [ ]:
vocab_low = build_vocab(df['processed_tweet'])

In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)

Glove : 
Found embeddings for 48.31% of vocab
Found embeddings for  83.05% of all text


In [ ]:
df['Tweet'] = df['processed_tweet']

In [ ]:
def data_prep(df):
    print("Splitting dataframe with shape {} into training and test datasets".format(df.shape))
    X_train, X_test  = train_test_split(df, test_size=0.1, random_state=2019)
    y_train, y_test = X_train['sentimen'].values, X_test['sentimen'].values
    
    print("Filling missing values")
    X_train = X_train['Tweet'].fillna('_NA_').values
    X_test = X_test['Tweet'].fillna('_NA_').values
    # X_submission = df_test['Tweet'].fillna('_NA_').values
    
    print("Tokenizing {} Tweet into words".format(df.shape[0]))
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X_train))
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    # X_submission = tokenizer.texts_to_sequences(X_submission)
    
    print("Padding sequences for uniform dimensions")
    X_train = pad_sequences(X_train, maxlen=maxlen)
    X_test = pad_sequences(X_test, maxlen=maxlen)
    # X_submission = pad_sequences(X_submission, maxlen=maxlen)
    
    print("Completed data preparation, returning training, test and submission datasets, split as dependent(X) and independent(Y) variables")
    # return X_train, X_test, y_train, y_test, X_submission
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = data_prep(df)

Splitting dataframe with shape (10806, 3) into training and test datasets
Filling missing values
Tokenizing 10806 Tweet into words
Padding sequences for uniform dimensions
Completed data preparation, returning training, test and submission datasets, split as dependent(X) and independent(Y) variables


In [ ]:
# embeddings_index = {}
# f = open(os.path.join('glove_wiki_id_50.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

Found 212550 word vectors.


In [ ]:
# EMBEDDING_DIM = 50
# embedding_matrix = np.zeros((len(embed_glove) + 1, EMBEDDING_DIM))
# for word, i in embed_glove.items():
#     print (i)
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

[ 0.015828  0.000282  0.0022    0.007403  0.004672  0.015625  0.006509
  0.013281  0.009538  0.007392 -0.001099 -0.003191 -0.013885  0.011943
  0.011973 -0.004197  0.007191  0.005189 -0.012265  0.0101   -0.002079
  0.000253  0.008111  0.006362 -0.001157  0.013243  0.000835 -0.007282
  0.005522  0.009601  0.008059 -0.006037  0.01209   0.002687 -0.004104
 -0.003712  0.008948  0.005749 -0.011684 -0.005787  0.000122  0.008251
  0.009211  0.007168  0.004828  0.002199  0.012714  0.016136  0.009771
 -0.01227 ]


IndexError: ignored

In [ ]:
embed_size = 50 # how big is each word vector
max_features = 212594 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a tweet to use

list_weight = [[0,0,0,0,0,0,0,0],[1,1,1,1,1,1,1]]

model1 = Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen, weights = list_weight))
model1.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.summary()

ValueError: ignored

In [ ]:
print(embed_glove['dan'])

[ 0.015828  0.000282  0.0022    0.007403  0.004672  0.015625  0.006509
  0.013281  0.009538  0.007392 -0.001099 -0.003191 -0.013885  0.011943
  0.011973 -0.004197  0.007191  0.005189 -0.012265  0.0101   -0.002079
  0.000253  0.008111  0.006362 -0.001157  0.013243  0.000835 -0.007282
  0.005522  0.009601  0.008059 -0.006037  0.01209   0.002687 -0.004104
 -0.003712  0.008948  0.005749 -0.011684 -0.005787  0.000122  0.008251
  0.009211  0.007168  0.004828  0.002199  0.012714  0.016136  0.009771
 -0.01227 ]


In [ ]:
print(embed_glove.values())

===============================================

In [ ]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove_wiki_id_50.txt', encoding="utf8")

In [ ]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
unique_words = set(all_words)
print(len(unique_words))

45


In [ ]:
vocab_length = 100
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[11, 48, 61, 81, 47], [7, 79, 43, 31, 15, 79, 13], [4, 6, 28, 13, 48, 54], [67, 54], [80, 74, 63, 12, 15, 79, 13], [26, 49, 31, 61], [55, 15, 43, 49, 64, 47], [13, 48, 49, 34, 47], [6, 87], [89, 12, 42], [19, 53], [13, 43, 55, 21], [15, 95, 98, 79, 7, 47], [7, 47, 43, 6], [15, 44, 98, 41], [7, 87, 48, 19]]


In [ ]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [ ]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[11 48 61 81 47  0  0]
 [ 7 79 43 31 15 79 13]
 [ 4  6 28 13 48 54  0]
 [67 54  0  0  0  0  0]
 [80 74 63 12 15 79 13]
 [26 49 31 61  0  0  0]
 [55 15 43 49 64 47  0]
 [13 48 49 34 47  0  0]
 [ 6 87  0  0  0  0  0]
 [89 12 42  0  0  0  0]
 [19 53  0  0  0  0  0]
 [13 43 55 21  0  0  0]
 [15 95 98 79  7 47  0]
 [ 7 47 43  6  0  0  0]
 [15 44 98 41  0  0  0]
 [ 7 87 48 19  0  0  0]]


In [ ]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 7, 20)             2000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 141       
Total params: 2,141
Trainable params: 2,141
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(padded_sentences, sentiments, epochs=2, verbose=1)

Epoch 1/2
1/1 [==============================] - 0s 486ms/step - loss: 0.6892 - acc: 0.7500
Epoch 2/2
1/1 [==============================] - 0s 12ms/step - loss: 0.6854 - acc: 0.7500


In [ ]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 75.000000


In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [ ]:
vocab_length = len(word_tokenizer.word_index) + 1

In [ ]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


In [ ]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove_wiki_id_50.txt', encoding="utf8")

In [ ]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_length, 50))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 50, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 7, 50)             2200      
_________________________________________________________________
flatten_2 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 351       
Total params: 2,551
Trainable params: 351
Non-trainable params: 2,200
_________________________________________________________________
None


In [ ]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 446ms/step - loss: 0.6979 - acc: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6957 - acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6936 - acc: 0.5625
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6914 - acc: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6893 - acc: 0.5625
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6872 - acc: 0.5625
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6851 - acc: 0.5625
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6830 - acc: 0.5625
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6809 - acc: 0.5625
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6788 - acc: 0.6250
Epoch 11/100
1/1 [==============================] - 0s 10ms/step - loss: 0.

In [ ]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.750000


In [ ]:
from keras.models import Model
from keras.layers import Input

deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 50, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A
flatten = Flatten()(embedding)
hidden = Dense(1, activation='sigmoid')(flatten)
model = Model(inputs=deep_inputs, outputs=hidden)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
embedding_23 (Embedding)     (None, 7, 50)             2200      
_________________________________________________________________
flatten_3 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 351       
Total params: 2,551
Trainable params: 351
Non-trainable params: 2,200
_________________________________________________________________
None


In [ ]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/100
1/1 [==============================] - 0s 441ms/step - loss: 0.6809 - acc: 0.6875
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6788 - acc: 0.6875
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6768 - acc: 0.6875
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6747 - acc: 0.6875
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6727 - acc: 0.6875
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6706 - acc: 0.8125
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6686 - acc: 0.8125
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6666 - acc: 0.8125
Epoch 9/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6646 - acc: 0.8125
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6626 - acc: 0.8125
Epoch 11/100
1/1 [==============================] - 0s 12ms/step - loss: 0.66

In [ ]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.078951    0.147156   -0.03935    ... -0.025625    0.053181
   0.012908  ]
 [ 0.105143   -0.211925    0.011668   ... -0.067152    0.14919201
  -0.19564   ]
 ...
 [ 0.095798    0.003681    0.077189   ...  0.030864    0.00545
  -0.014658  ]
 [ 0.051755   -0.023189   -0.069458   ... -0.005546    0.052251
  -0.060099  ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
